In [9]:
import collections
import random
import sys
import time

import simpy
from faker import Faker

In [10]:
NUM_BOLETO = 120
TIEMPO_SIMULACION = 120

In [11]:
acciones_globales = {}
acciones_individuales = {}
clientes = []
faker = Faker()


class Cliente:
    renegado: bool = False
    pelicula: str = 'NA'
    cantidad_boletos: int = 0
    acciones: dict = {}

    def __init__(self, nombre):
        self.nombre = nombre
        pass

    def __str__(self):
        return f'nombre: {self.nombre}, renegado {self.renegado}, pelicula {self.pelicula}, cantidad_boletos {self.cantidad_boletos}, acciones {self.acciones}'

In [12]:
def ventaBoletos(env, num_boletos, pelicula, teatro, name):
    with teatro.contador.request() as turno:
        resultado = yield turno | teatro.sold_out[pelicula]

        if turno not in resultado:
            acciones_individuales.get(name).append(('sin_turno', env.now))
            teatro.num_renegados[pelicula] += 1
            return

        if teatro.num_boletos[pelicula] < num_boletos:
            yield env.timeout(0.5)
            acciones_individuales.get(name).append(('renegado', env.now))
            return
        teatro.num_boletos[pelicula] -= num_boletos
        acciones_individuales.get(name).append(('logrado', env.now))
        if teatro.num_boletos[pelicula] < 2:
            acciones_globales['fin_boletos' + pelicula] = env.now
            teatro.sold_out[pelicula].succeed()
            teatro.tiempo_agotado[pelicula] = env.now
            teatro.num_boletos[pelicula] = 0
        yield env.timeout(1)


def llegadaClientes(env, teatro):
    while True:
        name = faker.name()
        acciones_individuales[name] = []

        yield env.timeout(random.expovariate(1 / 0.5))
        pelicula = random.choice(teatro.peliculas)
        num_boletos = random.randint(1, 6)

        acciones_individuales.get(name).append(('llega', env.now))
        acciones_individuales.get(name).append(('pelicula', pelicula))
        acciones_individuales.get(name).append(('num_boletos', num_boletos))

        if teatro.num_boletos[pelicula]:
            env.process(ventaBoletos(env, num_boletos, pelicula, teatro, name))
        else:
            acciones_individuales.get(name).append(('renegado', env.now))


Teatro = collections.namedtuple('Teatro', 'contador, peliculas, num_boletos, sold_out, tiempo_agotado, num_renegados')

print('Teatro Carlos Crespi - UPS')
env = simpy.Environment()

contador = simpy.Resource(env, capacity=1)
peliculas = ['Conjuro 3', 'Rapidos y Furiosos 10', 'Pulp Fiction']
num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
sold_out = {pelicula: env.event() for pelicula in peliculas}
tiempo_agotado = {pelicula: None for pelicula in peliculas}
num_renegados = {pelicula: 0 for pelicula in peliculas}

teatro = Teatro(contador, peliculas, num_boletos, sold_out, tiempo_agotado, num_renegados)
env.process(llegadaClientes(env, teatro))
env.run(until=TIEMPO_SIMULACION)

# Analisis y resultados
for pelicula in peliculas:
    if teatro.sold_out[pelicula]:
        print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' % (
        pelicula, teatro.tiempo_agotado[pelicula]))
        print('Numero de personas que salieron de la fila/renegados %s' % teatro.num_renegados[pelicula])

Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 91.8 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 23
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 96.3 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 26
Pelicula: Pulp Fiction se agoto en el tiempo 105.3 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 31


In [13]:
import pygame

In [37]:
import pygame

class TicketShop(pygame.sprite.Sprite):
    def __init__(self):
        super(TicketShop, self).__init__()
        self.surf = pygame.image.load('ticketshop.jpg').convert()
        self.rect = self.surf.get_rect(
            center=(750, 300,)
        )
        self.speed = 1

class Sprite(pygame.sprite.Sprite):
    def __init__(self, estado):
        self.estado = estado
        super(Sprite, self).__init__()
        self.surf = pygame.image.load('sprite.png').convert()
        self.rect = self.surf.get_rect(
            center=(-70, 300,)
        )
        self.speed = 1

    def update(self):

        if self.rect.y == 0 or self.rect.y > 800:
            self.kill()

        if self.estado == 'sin_turno' and self.rect.right > 200:
            self.rect.move_ip(1, 1)
            return
        elif self.estado == 'renegado' and self.rect.right > 700:
            self.surf = pygame.image.load('angry.png').convert()
            self.rect.move_ip(0, 1)
            return
        elif self.estado == 'logrado' and self.rect.right > 700:
            self.surf = pygame.image.load('ticket.png').convert()
            self.rect.move_ip(0, -1)
            return
        self.rect.move_ip(1,0)


pygame.init()

screen = pygame.display.set_mode((800, 600))
ADDCLIENT = pygame.USEREVENT + 1
pygame.time.set_timer(ADDCLIENT, 650)

all_sprites = pygame.sprite.Group()
running = True
clock = pygame.time.Clock()


elements = list(acciones_individuales.values())
index = 0

while running:

    if pygame.event.get(pygame.QUIT): break

    for event in pygame.event.get():
        if event.type == ADDCLIENT and len(elements[index]) > 0:
            new_sprite = Sprite(elements[index][3][0])
            all_sprites.add(new_sprite)
            all_sprites.add(TicketShop())
            index+=1

    all_sprites.update()
    screen.fill((0, 0, 0))

    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)

    pygame.display.flip()
    clock.tick(120)
pygame.quit()